In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import os
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm
sns.set_palette('Dark2')
sns.set_style({'axes.axisbelow': True, 'axes.edgecolor': '.15', 'axes.facecolor': 'white',
               'axes.grid': True, 'axes.labelcolor': '.15', 'axes.linewidth': 1.25, 
               'figure.facecolor': 'white', 'font.family': ['sans-serif'], 'grid.color': '.15',
               'grid.linestyle': ':', 'grid.alpha': .5, 'image.cmap': 'Greys', 
               'legend.frameon': False, 'legend.numpoints': 1, 'legend.scatterpoints': 1,
               'lines.solid_capstyle': 'round', 'axes.spines.right': False, 'axes.spines.top': False,  
               'text.color': '.15',  'xtick.top': False, 'ytick.right': False, 'xtick.color': '.15',
               'xtick.direction': 'out', 'xtick.major.size': 6, 'xtick.minor.size': 3,
               'ytick.color': '.15', 'ytick.direction': 'out', 'ytick.major.size': 6,'ytick.minor.size': 3})
sns.set_context('paper')

#http://phyletica.org/matplotlib-fonts/
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import matplotlib.gridspec as gridspec

In [3]:
from snapanalysis.config import OUTPUT_DIRECTORY as MAIN_OUTPUT_DIRECTORY
OUTPUT_DIRECTORY = os.path.join(MAIN_OUTPUT_DIRECTORY, 'ptm-response')

if not os.path.isdir(OUTPUT_DIRECTORY):
    os.makedirs(OUTPUT_DIRECTORY)
    
OUTPUT_DIRECTORY_BARPLOTS = os.path.join(OUTPUT_DIRECTORY, 'barplots')

if not os.path.isdir(OUTPUT_DIRECTORY_BARPLOTS):
    os.makedirs(OUTPUT_DIRECTORY_BARPLOTS)

In [4]:
from snapanalysis.models.ptm_response.main import OUTPUT_FILE as PTM_RESPONSE_FILE
with pd.HDFStore(PTM_RESPONSE_FILE, 'r') as store:
    
    enrichment_complexes = store['ptm_stats/joint_camera_complexes']

In [5]:
import importlib
import helpers
importlib.reload(helpers)
from helpers import *

In [6]:
import urllib

print('Informative URIs for predictors:')
print()

for predictor in PREDICTOR_ORDER:
    
    uri = predictor_sorted_uri(predictor)
    
    print(f'{predictor:>20}: {uri}')

Informative URIs for predictors:

               H2A.Z: http://ife-snap-data/proteins?pdorder=H36,H26,H37,H43,H45&showsimilar=false&noclusterproteins=true
                H3ac: http://ife-snap-data/proteins?pdorder=H05,H10,H02,H12,H16,H20,H21,H25,H22,H26,H41,H43&showsimilar=false&noclusterproteins=true
             H3K4me1: http://ife-snap-data/proteins?pdorder=H05,H03,H06,H13,H16,H30,H31,H15,H09&showsimilar=false&noclusterproteins=true
             H3K4me3: http://ife-snap-data/proteins?pdorder=H02,H13,H21,H15,H11,H14,H22,H40,H41&showsimilar=false&noclusterproteins=true
         H3K9acK14ac: http://ife-snap-data/proteins?pdorder=H15,H05,H09,H02,H11,H16,H18,H06,H19,H21,H23,H22,H24,H41,H42&showsimilar=false&noclusterproteins=true
             H3K9me2: http://ife-snap-data/proteins?pdorder=H47,H27M,H47M&showsimilar=false&noclusterproteins=true
             H3K9me3: http://ife-snap-data/proteins?pdorder=H01,H27M,H01M,H03,H04,H03M,H04M&showsimilar=false&noclusterproteins=true
             

In [7]:
color_me1 = '#BBE1BB'
color_me2 = '#9FBA7D'
color_me3 = '#6F9979'
color_ac = '#6484A2'

palette = {
     'H2A.Z': '#786D9B',
     'H3K4me1': color_me1,
     'H3K4me3': color_me3,
     'H3K9acK14ac': color_ac,
     'H3K9me2': color_me2,
     'H3K9me3': color_me3,
     'H3K27ac': color_ac,
     'H3K27me2': color_me2,
     'H3K27me3': color_me3,
     'H3ac': color_ac,
     'H4K16ac': color_ac,
     'H4K20me2': color_me2,
     'H4K20me3': color_me3,
     'H4ac': color_ac,
     'DNA Methylation': '#85B3B2',
}




In [8]:
GROUPED_ORDER = [
    ['H2A.Z', 'DNA Methylation'],
    
    ['H3K4me1', 'H3K4me3'],
    ['H3ac', 'H3K9acK14ac', 'H3K27ac'],
    
    ['H3K9me2', 'H3K9me3'],
    ['H3K27me2', 'H3K27me3'],
    
    ['H4ac', 'H4K16ac'],
    ['H4K20me2', 'H4K20me3']
]

In [9]:
limma_results = load_limma_data()

In [10]:
complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

In [11]:
enrichment_complexes

NGenes Direction  \
predictor Complex                                                               
H3K9me3   ncPRC1.6 (exclusive subunits)                         6.0        Up   
          ncPRC1.6                                             15.0        Up   
          SIN3A/B (exclusive subunits)                          9.0        Up   
          PRC1                                                  9.0        Up   
          CAF-1                                                 3.0        Up   
          BHC                                                   8.0        Up   
          HUSH                                                  4.0        Up   
          SIN3A/B                                              18.0        Up   
          KAP1-SETDB1-DNMT1-ZNF304                              3.0        Up   
          SF3B                                                  7.0      Down   
          INO80 (exclusive subunits)                           11.0        Up   
          KAP1-HP1                                              3.0        Up   
          B-MYB-MuvB                                            6.0        Up   
          SRCAP                                                 9.0      Down   
          ORC                                                   5.0        Up   
          B-MYB-MuvB (exclusive subunits)                       5.0        Up   
          ASAP                                                  6.0      Down   
          Integrator (exclusive subunits)                      13.0        Up   
          THO                                                   6.0      Down   
          Integrator                                           14.0        Up   
          PRC1 (exclusive subunits)                             6.0        Up   
          PR-DUB                                                5.0        Up   
          ncPRC1                                                5.0        Up   
          TREX                                                  9.0      Down   
          40S Ribosomal subunit                                29.0      Down   
          ncPRC1.1                                             11.0        Up   
          INO80                                                15.0        Up   
          Mediator                                             17.0        Up   
          NuA4 (exclusive subunits)                             6.0        Up   
          PRC2.1                                               12.0        Up   
...                                                             ...       ...   
H3ac      Mitochondrial respiratory chain complex I             4.0        Up   
          ncPRC1.1 (exclusive subunits)                         5.0        Up   
          CKM                                                   3.0        Up   
          mCRD-poly(A)-bridging complex                         3.0      Down   
          TFIID (exclusive subunits)                            4.0        Up   
          RNA Pol II (exclusive subunits)                       6.0        Up   
          NSL (exclusive subunits)                              3.0        Up   
          Base Excision Repair Complex (exclusive subunits)     3.0      Down   
          VHL SCF-like                                          4.0        Up   
          XPC (exclusive subunits)                              3.0        Up   
          Fanconi Anemia                                        4.0        Up   
          PRC2.1 (exclusive subunits)                           5.0        Up   
          RNA Pol II                                           11.0        Up   
          B-MYB-MuvB (exclusive subunits)                       5.0        Up   
          CRL4-DDB2                                             4.0        Up   
          CDCA5-PDS5-Cohesin                                    3.0        Up   
          VCB                                                   3.0        Up   
          S

In [12]:
import re

complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

max_proteins = 25

for complex_ in tqdm(complexes_to_plot):
    
    # Title of plot
    name = complex_
    complex_filename = re.sub('[^a-zA-Z0-9]+', '_', complex_).lower()
    filename = f'barplot-{complex_filename}.pdf'
    
    print(complex_)
    
    proteins = members_of(complex_)
    
    if len(proteins) > max_proteins:
        print(f'Skipping {complex_} because it has more than {max_proteins} proteins')
        continue
        
    df = limma_results.loc(axis=0)[:, proteins]
    
    if (df.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_} because it does not match quality criteria')
        continue
   

    # PLOT
    fig = plt.figure(figsize=(5, 6.45), 
                     constrained_layout=True)

    n_groups = len(GROUPED_ORDER)

    spec = gridspec.GridSpec(ncols=1,
                             nrows=n_groups, 
                             figure=fig,
                             height_ratios=[len(x) for x in GROUPED_ORDER])

    ax = None
    for i, predictor_subgroup in enumerate(GROUPED_ORDER):
        ax = fig.add_subplot(spec[i, :], sharex=ax)

        medians = []
        ci = []


        subgroup_df = df.loc[predictor_subgroup]
        
        significant_predictors = set()

        for predictor in predictor_subgroup:
            
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians.append(mu)
            ci.append([err_low, err_high])


        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians, ci)):
            ax.barh(y, mu, 
                    xerr=np.atleast_2d(ci).T,
                    color=palette[predictor],
                    error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                    edgecolor='black', linewidth=1.0)
        
     
        for significant, subsubgroup_df in subgroup_df.groupby('significant'):
        
            sns.stripplot(y='predictor', x='logFC',
                          hue='predictor',
                          hue_order=predictor_subgroup,
                          dodge=False,
                          palette=palette,
                          order=predictor_subgroup,
                          size=5,
                          alpha=.8,
                          edgecolor='black',
                          linewidth=1.0,
                          jitter=0.25,
                          marker='o' if significant else 'X',
                          ax=ax, data=subsubgroup_df.reset_index())

    #     ax.axvline(0, color='#666666')
        ax.legend_.set_visible(False)

        ax.set_ylabel('')
        
        ax.yaxis.set_tick_params(length=0, labelsize=10)
        ax.xaxis.set_tick_params(length=0)
        
        for predictor, tick in zip(predictor_subgroup, ax.get_yticklabels()):
            if predictor in significant_predictors:
                tick.set_fontweight('bold')

        if i != n_groups-1:
            sns.despine(left=True, bottom=True, offset=5, ax=ax)
            for tic in ax.xaxis.get_major_ticks():
                tic.tick1On = tic.tick2On = False
                tic.label1On = tic.label2On = False
            ax.set_xlabel('')
        else:
            sns.despine(left=True, bottom=False, offset=5, ax=ax)
            ax.set_xlabel('Change to H/L ratio attributed to modification', 
                          fontsize=10)

        if i == 0:
            ax.set_title(name, fontsize=10)


    fig.savefig(os.path.join(OUTPUT_DIRECTORY_BARPLOTS, filename), 
                bbox_inches='tight')
    plt.close()
#     break


  0%|          | 0/96 [00:00<?, ?it/s]

40S Ribosomal subunit
Skipping 40S Ribosomal subunit because it has more than 25 proteins
AP-2 dimers


  2%|▏         | 2/96 [00:02<02:16,  1.45s/it]

APC/C


  3%|▎         | 3/96 [00:05<02:45,  1.78s/it]

...

 98%|█████████▊| 94/96 [02:41<00:03,  1.79s/it]

ncPRC1.6


 99%|█████████▉| 95/96 [02:43<00:01,  1.86s/it]

ncPRC1.6 (exclusive subunits)


100%|██████████| 96/96 [02:44<00:00,  1.80s/it]


In [13]:
[c for c in complexes_to_plot if 'ino80' in c.lower()]

['INO80', 'INO80 (exclusive subunits)']

In [14]:
import re

complexes_to_plot = sorted(enrichment_complexes.reset_index().query('significant')['Complex'].unique())

max_proteins = 25

for complex_ in tqdm(complexes_to_plot):
    
    # Title of plot
    name = complex_
    complex_filename = re.sub('[^a-zA-Z0-9]+', '_', complex_).lower()
    filename = f'barplot-horizontal-{complex_filename}.pdf'
    
    print(complex_)
    
    proteins = members_of(complex_)
    
    if len(proteins) > max_proteins:
        print(f'Skipping {complex_} because it has more than {max_proteins} proteins')
        continue
        
    df = limma_results.loc(axis=0)[:, proteins]
    
    if (df.groupby(level='predictor').size() > 2).sum() < 10:
        print(f'Skipping {complex_} because it does not match quality criteria')
        continue
   

    # PLOT
    fig = plt.figure(figsize=(6.25, 4.3), 
                     constrained_layout=True)

    n_groups = len(GROUPED_ORDER)

    spec = gridspec.GridSpec(ncols=n_groups,
                             nrows=1, 
                             figure=fig,
                             width_ratios=[len(x) for x in GROUPED_ORDER])

    ax = None
    for i, predictor_subgroup in enumerate(GROUPED_ORDER):
        ax = fig.add_subplot(spec[:, i], sharey=ax)

        medians = []
        ci = []


        subgroup_df = df.loc[predictor_subgroup]
        
        significant_predictors = set()

        for predictor in predictor_subgroup:
            
            try:
                subdf = subgroup_df.loc[predictor]
                subdf_enrichment = enrichment_complexes.loc[predictor, complex_]
            except KeyError:
                medians.append(0)
                ci.append([0, 0])
                
                continue
            
            
            mu = subdf_enrichment['empirical_median']
            ci_low = subdf_enrichment['empirical_median_ci_left']
            ci_high = subdf_enrichment['empirical_median_ci_right']
            
            if subdf_enrichment['significant']:
                significant_predictors.add(predictor)
            
            err_low = mu - ci_low
            err_high = ci_high - mu

            medians.append(mu)
            ci.append([err_low, err_high])


        for y, (predictor, mu, ci) in enumerate(zip(predictor_subgroup, medians, ci)):
            ax.bar(y, mu, 
                   yerr=np.atleast_2d(ci).T,
                   color=palette[predictor],
                   error_kw=dict(elinewidth=3, capsize=5, capthick=2),
                   edgecolor='black', linewidth=1.0)
        
     
        for significant, subsubgroup_df in subgroup_df.groupby('significant'):
        
            sns.stripplot(x='predictor', y='logFC',
                          hue='predictor',
                          hue_order=predictor_subgroup,
                          dodge=False,
                          palette=palette,
                          order=predictor_subgroup,
                          size=5,
                          alpha=.8,
                          edgecolor='black',
                          linewidth=1.0,
                          jitter=0.25,
                          marker='o' if significant else 'X',
                          ax=ax, data=subsubgroup_df.reset_index())

    #     ax.axvline(0, color='#666666')
        ax.legend_.set_visible(False)

        ax.set_xlabel('')
        
        ax.xaxis.set_tick_params(length=0, labelsize=10, rotation=90)
        ax.yaxis.set_tick_params(length=0)
        
        for predictor, tick in zip(predictor_subgroup, ax.get_xticklabels()):
            if predictor in significant_predictors:
                tick.set_fontweight('bold')

        if i != 0:
            sns.despine(left=True, bottom=True, offset=5, ax=ax)
            for tic in ax.yaxis.get_major_ticks():
                tic.tick1On = tic.tick2On = False
                tic.label1On = tic.label2On = False
            ax.set_ylabel('')
        else:
            sns.despine(left=False, bottom=True, offset=5, ax=ax)
            ax.set_ylabel('Change to H/L ratio attributed to modification', 
                          fontsize=10)

    fig.suptitle(name, fontsize=10)


    fig.savefig(os.path.join(OUTPUT_DIRECTORY_BARPLOTS, filename), 
                bbox_inches='tight')
    plt.close()
#     break


  0%|          | 0/96 [00:00<?, ?it/s]

40S Ribosomal subunit
Skipping 40S Ribosomal subunit because it has more than 25 proteins
AP-2 dimers


  2%|▏         | 2/96 [00:02<01:40,  1.07s/it]

APC/C


  3%|▎         | 3/96 [00:06<03:15,  2.10s/it]

...

 98%|█████████▊| 94/96 [03:08<00:03,  1.77s/it]

ncPRC1.6


 99%|█████████▉| 95/96 [03:09<00:01,  1.79s/it]

ncPRC1.6 (exclusive subunits)


100%|██████████| 96/96 [03:11<00:00,  1.75s/it]
